In [ ]:
import os
from pathlib import Path

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Building calibration

Building simulation are useful for comparing variants during a building design phase. Simulation results are compared in order to select the best building materials or HVAC system regarding various indicators (energy consumption, thermal comfort, etc.)

But when we compare the simulation outputs to measured values big differences are usually observed. There are a lots of explanations to this gap. But usually, "errors" can be classified in 3 categories:
- **Boundary conditions** : meteo file used in simulation (external temperature, solar radiation, wind, etc), strongly differs from actual meteo.
- **Hypothesis** parameter values or schedules are different from what we estimated. Deteriorated Insulation material, inefficient HVAC system, high temperature set point, etc.
- **Models** a poor choice of physical model or ignored physical phenomenon (natural ventilation, thermal zone separation, etc.)

In a building calibration process, the objective is to